# Simple Chat

Awesome! We now know how to connect to our LLM and ask stuff from models!

Now that we can do basic communication with our LLMs, we'll start on building a complex agent. Let's start with some basic configurations, learning our syntax, and establishing basic communication with an LLM through LangGraph, then we'll complicate it by adding some tools the LLM can take advantage of.

Could we build a simple chat without LangGraph? Yes, easily. Could we do the rest of the course without LangGraph? Yes, but not easily.

## 1 Configs

### 1.1 Installs

* `dotenv` – loads and manages environment variables from a `.env` file.

* `langchain-core` – the lightweight core of LangChain, providing the `Runnable` interfaces and core abstractions.

* `langgraph` – builds stateful agent workflows as directed graphs of nodes and edges.

* `requests` – simple HTTP client for making API calls (e.g., to Databricks endpoints).

In [1]:
%%capture
%pip install \
  dotenv \
  "langchain-core==0.3.79" \
  "langgraph==0.2.41" \
  requests

In [ ]:
# restart kernel to use newly installed packages
%reset -f

Don't know how to reset  #, please run `%reset?` for details
Don't know how to reset  vscode, please run `%reset?` for details


### 1.2 Imports

 - from `langgraph.graph import StateGraph, END`
   - `StateGraph` - build and compile our node graph.
   - `END` - sentinel that tells LangGraph where to stop.

 - `from langchain_core.runnables import RunnableLambda` - wraps a normal Python function so the graph can call it like any other LangChain “runnable.”

 - `from typing import Dict, List, Optional, TypedDict` - creates AgentState, a typed dictionary that documents (and type-checks) the keys we pass between nodes.

 - `my_functions` - Our custom useful functions

 - `os` - Access environment variables that store our Azure configuration

 - `dotenv.load_dotenv` - Reads the `.env` file and safely loads the environmental variables

In [3]:
# LangGraph core components for building agent workflows
from langgraph.graph import StateGraph, END       # StateGraph: main graph builder, END: termination signal
from langchain_core.runnables import RunnableLambda  # Converts functions to graph-compatible nodes

# Type annotations for better code clarity and IDE support
from typing import Dict, List, Optional, TypedDict

from dotenv import load_dotenv
import os

# Our custom utility functions from the previous notebook
from my_functions import databricks_llm

### 1.3 Loading Environmental Variables

Load our Azure OpenAI configuration from the secure `.env` file.

In [ ]:
# Load environment variables (Azure endpoints, API versions, etc.)
load_dotenv('.env', override=True)

True

## 2 Defining Functions and Classes

This section establishes the core components of our LangGraph agent:
- **AgentState**: The data structure that flows between nodes
- **Agent Functions**: The actual logic that processes the state

### 2.1 Agent State Class

**The Heart of LangGraph: State Management**

The `AgentState` is a typed dictionary that carries information between all nodes in our graph. Think of it as a shared memory that gets passed around and updated by each agent/tool.

**Key Concepts:**
- **Persistence**: State persists across all nodes in the execution
- **Updates**: Each node can read from and write to the state
- **Type Safety**: TypedDict provides structure and IDE support
- **Scalability**: Easy to add new fields as your agent grows

**Current Fields:**
- `chat_history`: Complete conversation in OpenAI format
- `output`: Last response from the assistant (for easy access)

In [5]:
# Define the state structure that flows through our agent graph
class AgentState(TypedDict, total=False):
    """
    State container for our simple chat agent.
    
    This state object gets passed between all nodes in the graph,
    allowing them to share information and maintain conversation context.
    """
    chat_history: List[Dict[str, str]]  # Complete conversation in OpenAI format [{"role": "user", "content": "..."}]
    output: Optional[str]               # Most recent assistant response (for convenience)

### 2.2 Defining Agent Functions

**Converting Business Logic into Graph Nodes**

In LangGraph, every piece of functionality is a **node** - a function that:
1. **Receives**: The current AgentState
2. **Processes**: Performs some operation (LLM call, tool usage, logic)
3. **Returns**: Updated AgentState with new information

**Function Pattern:**
- Input: `state` (AgentState dictionary)
- Output: Modified `state` with updates
- Side effects: Printing, external API calls, etc.

**The chat_agent function below:**
- Takes the conversation history from state
- Sends it to our LLM
- Updates the conversation with the AI's response
- Returns the enhanced state

In [6]:
def chat_agent(state):
    """
    Core chat agent node: sends conversation to LLM and updates state with response.
    
    This function represents our main AI agent behavior:
    1. Takes current conversation history from state
    2. Sends it to the LLM via our openai_llm function
    3. Updates the conversation history with the AI's response
    4. Stores the response in 'output' for easy access
    
    Args:
        state (AgentState): Current state containing chat history
    
    Returns:
        AgentState: Updated state with new AI response added
    """
    print("\n--- CHAT AGENT NODE ---")
    print(f"Processing conversation with {len(state['chat_history'])} messages")

    # === CALL THE LLM ===
    # Send current conversation history to our LLM function
    # This function handles Azure authentication and API calls
    reply = databricks_llm(
        state["chat_history"],  # Pass the complete conversation
        os.getenv("CHAT_ENDPOINT")   # Model deployment name
    )
    
    print(f"LLM Response: {reply[:100]}...")  # Show first 100 chars

    # === UPDATE THE STATE ===
    # Add the AI's response to the conversation history
    state["chat_history"].append({"role": "assistant", "content": reply})
    
    # Store the response in output field for easy access
    state["output"] = reply

    print(f"Updated conversation now has {len(state['chat_history'])} messages")
    print("--- CHAT AGENT NODE END ---\n")

    # Return the updated state (this gets passed to the next node, or back to caller)
    return state

## 3 Building the Agent Graph

Now we assemble our agent components into a working LangGraph workflow.

### 3.1 Graph Definition

**Understanding LangGraph Architecture**

A LangGraph is like a flowchart for AI agents, consisting of:

**Core Components:**
- **Nodes**: Functions that process the state (our `chat_agent` function)
- **Edges**: Connections that determine the flow between nodes
- **Entry Point**: Which node runs first when the graph is invoked
- **END**: Special signal that stops execution

**Our Simple Graph Flow:**
```
START → chat_agent → END
```

**Why Use Graphs?**
- **Modularity**: Each node has a single responsibility
- **Debugging**: Easy to trace execution flow
- **Scalability**: Simple to add new nodes (tools, validators, routers)
- **Conditional Logic**: Can add conditional edges based on state
- **Error Handling**: Built-in retry and error recovery patterns

In [7]:
# === STEP 1: INITIALIZE THE GRAPH ===
# Create a StateGraph that will manage our AgentState
g = StateGraph(AgentState)
print("✓ Graph initialized with AgentState structure")

# === STEP 2: ADD NODES ===
# Convert our Python function into a graph node using RunnableLambda
# The node name "chat_agent" is how we'll reference it in edges
g.add_node("chat_agent", RunnableLambda(chat_agent))
print("✓ Added 'chat_agent' node")

# === STEP 3: DEFINE ENTRY POINT ===
# When someone calls .invoke() on this graph, start with the chat_agent node
g.set_entry_point("chat_agent")
print("✓ Set 'chat_agent' as entry point")

# === STEP 4: DEFINE EDGES (FLOW CONTROL) ===
# After chat_agent finishes, go to END (stop execution)
# This creates a simple linear flow: START → chat_agent → END
g.add_edge("chat_agent", END)
print("✓ Added edge: chat_agent → END")

# === STEP 5: COMPILE THE GRAPH ===
# Convert the graph definition into an executable workflow
simple_chat = g.compile()
print("✓ Graph compiled successfully!")

print("\nGraph structure:")
print("START → chat_agent → END")
print("\nThe graph is ready to process conversations!")

✓ Graph initialized with AgentState structure
✓ Added 'chat_agent' node
✓ Set 'chat_agent' as entry point
✓ Added edge: chat_agent → END
✓ Graph compiled successfully!

Graph structure:
START → chat_agent → END

The graph is ready to process conversations!


### 3.2 Interactive Chat Loop

**Putting It All Together**

This chat loop demonstrates how to use our compiled graph in practice:

1. **Initialize**: Create starting state with system prompt
2. **Loop**: Continuously get user input and process through graph
3. **Update**: Add user messages to conversation history
4. **Execute**: Run the graph with `.invoke(state)`
5. **Display**: Show the AI's response to the user

**Key Features:**
- **Persistent Memory**: Full conversation history maintained
- **System Prompt**: AI personality set with initial system message
- **Exit Strategy**: Type "exit" to quit the conversation
- **State Management**: Graph handles all state updates automatically

In [8]:
# === INITIALIZE CONVERSATION STATE ===
# Start with a system prompt to define the AI's personality/behavior
chat_history = [
    {"role": "system", "content": "You are a helpful AI assistant that always talks like a pirate."}
]

# Create initial state object with conversation history and empty output
state = AgentState(
    chat_history=chat_history,
    output=None
)

print("🏴‍☠️ Pirate AI Assistant initialized!")
print("Type 'exit' to quit the conversation.\n")

# === MAIN CHAT LOOP ===
while True:
    # === GET USER INPUT ===
    user_text = input("You: ").strip()
    
    # === CHECK FOR EXIT COMMAND ===
    if user_text.lower() == "exit":
        print("Goodbye! May the winds fill your sails! 🏴‍☠️")
        break
    
    # Skip empty inputs
    if not user_text:
        continue
    
    print(f"User input received: '{user_text}'")
    
    # === ADD USER MESSAGE TO CONVERSATION ===
    # Append the user's message to our conversation history
    state["chat_history"].append({"role": "user", "content": user_text})
    print(f"Conversation now has {len(state['chat_history'])} messages")
    
    # === EXECUTE THE GRAPH ===
    # This runs our entire agent workflow:
    # 1. Calls chat_agent node with current state
    # 2. chat_agent sends conversation to LLM
    # 3. chat_agent updates state with AI response
    # 4. Returns updated state
    print("Invoking graph...")
    state = simple_chat.invoke(state)
    
    # === DISPLAY AI RESPONSE ===
    # The 'output' field contains the most recent AI response
    print(f"Assistant: {state['output']}")
    print("-" * 50)

🏴‍☠️ Pirate AI Assistant initialized!
Type 'exit' to quit the conversation.

User input received: 'hi'
Conversation now has 2 messages
Invoking graph...

--- CHAT AGENT NODE ---
Processing conversation with 2 messages
LLM Response: Yer lookin' fer a swashbucklin' conversation, eh? Alright then, let's set sail fer a grand adventure...
Updated conversation now has 3 messages
--- CHAT AGENT NODE END ---

Assistant: Yer lookin' fer a swashbucklin' conversation, eh? Alright then, let's set sail fer a grand adventure! What be bringin' ye to these fair waters?
--------------------------------------------------
Goodbye! May the winds fill your sails! 🏴‍☠️


## Summary: Simple Chat Agent

**What We Built:**
A basic but complete AI agent using LangGraph that maintains conversation context and responds with a pirate personality.

**Key Components:**
1. **AgentState**: Type-safe state container for conversation data
2. **chat_agent()**: Node function that processes conversations via LLM
3. **StateGraph**: Workflow that orchestrates the agent execution
4. **Chat Loop**: Interactive interface for users

**Why LangGraph?**
While we could build simple chat without LangGraph, it provides:
- **Structured Architecture**: Clear separation of concerns
- **Scalability**: Easy to add tools, validation, routing logic
- **Debugging**: Built-in execution tracing and state inspection
- **Error Handling**: Retry mechanisms and failure recovery
- **Complex Workflows**: Support for conditional logic and parallel execution

**Next Steps:**
In the following notebooks, we'll enhance this foundation by:
- Adding tools the agent can use (web search, calculations, etc.)
- Implementing conditional logic and routing
- Building multi-agent systems
- Adding memory and persistence

**Foundation Complete!** 🏴‍☠️
You now have a working AI agent framework that can be extended with virtually any functionality.